In [3]:
print("hello")

hello


In [1]:
import pandas as pd

# Load the CSV file
file_path = r"C:\Users\clint\Desktop\Scraping Task\combined_data.csv"
data = pd.read_csv(file_path)

# Display the first few rows of the data
data

,title,link,page,Month,Year,Date,Is_readily_available
0,COMMERCIAL TRUCK GUIDELINES,https://discover.jdpa.com/hubfs/Files/Industry...,1.0,Dec,2021,2021-12-01,0
1,COMMERCIAL TRUCK GUIDELINES,https://discover.jdpa.com/hubfs/Files/Industry...,1.0,Apr,2021,2021-04-01,0
2,COMMERCIAL TRUCK GUIDELINES,https://discover.jdpa.com/hubfs/Files/Industry...,1.0,Jun,2020,2020-06-01,0
3,COMMERCIAL TRUCK GUIDELINES,https://discover.jdpa.com/hubfs/Files/Industry...,1.0,Sep,2020,2020-09-01,0
4,COMMERCIAL TRUCK GUIDELINES,https://discover.jdpa.com/hubfs/Files/Industry...,1.0,Oct,2021,2021-10-01,0
...,...,...,...,...,...,...,...
110,NaN,https://cdn2.hubspot.net/hubfs/4239280/Files/I...,NaN,May,2018,2018-05-01,0
111,NaN,https://cdn2.hubspot.net/hubfs/4239280/Files/I...,NaN,April,2018,2018-04-01,0
112,NaN,https://cdn2.hubspot.net/hubfs/4239280/Files/I...,NaN,March,2018,2018-03-01,0
113,NaN,https://cdn2.hubspot.net/hubfs/4239280/Files/I...,NaN,February,2018,2018-02-01,0


In [2]:
import os
import requests
import pandas as pd
from urllib.parse import urlparse
from tqdm import tqdm
import time
import csv
from datetime import datetime

# Load your data - you need to specify your data source
# For example:
# data = pd.read_csv("your_data.csv")  
# or 
# data = pd.DataFrame({"link": ["http://example.com/file.pdf", "http://example.com/another.pdf"]})

# Define the output directory
output_dir = r"C:\Users\clint\Desktop\Scraping Task\pdfs"
# Create logs directory
logs_dir = r"C:\Users\clint\Desktop\Scraping Task\logs"

# Create the directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

# Create log files with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
non_pdf_log = os.path.join(logs_dir, f"non_pdf_links_{timestamp}.csv")
duplicate_log = os.path.join(logs_dir, f"duplicate_links_{timestamp}.csv")
error_log = os.path.join(logs_dir, f"error_links_{timestamp}.csv")

# Initialize log files with headers
with open(non_pdf_log, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['URL', 'Content-Type'])

with open(duplicate_log, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['URL'])

with open(error_log, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['URL', 'Error'])

# Function to download a PDF file
def download_pdf(url, output_path, timeout=30):
    try:
        # Send a GET request to the URL with timeout
        response = requests.get(url, stream=True, timeout=timeout, allow_redirects=True)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Check if the content is a PDF
        content_type = response.headers.get('Content-Type', '')
        if 'application/pdf' not in content_type and not url.lower().endswith('.pdf'):
            print(f"Link is not a PDF: {url} (Content-Type: {content_type})")
            # Log non-PDF link
            with open(non_pdf_log, 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow([url, content_type])
            return False
        
        # Write the content to a file
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        return True
    except requests.Timeout:
        error_msg = f"Timeout: Request took longer than {timeout} seconds"
        print(f"Timeout downloading {url}: {error_msg}")
        # Log error
        with open(error_log, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([url, error_msg])
        return False
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        # Log error
        with open(error_log, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([url, str(e)])
        return False

# Track downloaded URLs to avoid duplicates
downloaded_urls = set()
successful_downloads = 0
skipped_duplicates = 0
non_pdf_links = 0
error_links = 0

# Verify data is loaded before starting the loop
if 'data' not in locals() or len(data) == 0:
    raise ValueError("No data available. Please define the 'data' DataFrame before running this code.")

# Define a maximum runtime to prevent infinite loops (e.g., 1 hour)
max_runtime = 3600  # in seconds
start_time = time.time()

# Download PDFs from the DataFrame
for index, row in tqdm(data.iterrows(), total=len(data), desc="Downloading PDFs"):
    # Check if we've exceeded the maximum runtime
    if time.time() - start_time > max_runtime:
        print(f"Maximum runtime of {max_runtime/60:.1f} minutes reached. Stopping.")
        break
        
    if pd.notna(row['link']):
        url = row['link']
        
        # Skip if this URL has already been downloaded
        if url in downloaded_urls:
            skipped_duplicates += 1
            # Log duplicate link
            with open(duplicate_log, 'a', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow([url])
            continue
        
        # Generate a filename from the URL or use the index
        parsed_url = urlparse(url)
        filename = os.path.basename(parsed_url.path)
        
        # If filename is empty or doesn't end with .pdf, create a default name
        if not filename or not filename.lower().endswith('.pdf'):
            filename = f"document_{index}.pdf"
        
        # Check for filename collisions and add a suffix if needed
        base_name, extension = os.path.splitext(filename)
        counter = 1
        output_path = os.path.join(output_dir, filename)
        
        while os.path.exists(output_path):
            filename = f"{base_name}_{counter}{extension}"
            output_path = os.path.join(output_dir, filename)
            counter += 1
        
        # Download the PDF
        success = download_pdf(url, output_path)
        if success:
            downloaded_urls.add(url)
            successful_downloads += 1
        else:
            # Remove the file if it was created but is not a PDF
            if os.path.exists(output_path):
                os.remove(output_path)
            non_pdf_links += 1

# Create a summary log file
summary_log = os.path.join(logs_dir, f"download_summary_{timestamp}.csv")
with open(summary_log, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Metric', 'Count'])
    writer.writerow(['Successful Downloads', successful_downloads])
    writer.writerow(['Skipped Duplicates', skipped_duplicates])
    writer.writerow(['Non-PDF Links', non_pdf_links])
    writer.writerow(['Runtime (minutes)', f"{(time.time() - start_time)/60:.1f}"])

print(f"Downloaded {successful_downloads} PDFs.")
print(f"Skipped {skipped_duplicates} duplicate links.")
print(f"Disregarded {non_pdf_links} non-PDF links.")
print(f"Total runtime: {(time.time() - start_time)/60:.1f} minutes")
print(f"\nLog files created in {logs_dir}:")
print(f"- Non-PDF links: {os.path.basename(non_pdf_log)}")
print(f"- Duplicate links: {os.path.basename(duplicate_log)}")
print(f"- Error links: {os.path.basename(error_log)}")
print(f"- Summary: {os.path.basename(summary_log)}")

Link is not a PDF: https://www.jdpowervalues.com/article/september-2022-commercial-truck-guidelines (Content-Type: text/html; charset=UTF-8)


Link is not a PDF: https://www.jdpowervalues.com/article/class-8-pricing-still-sky-high-risks-intensify (Content-Type: text/html; charset=UTF-8)


Link is not a PDF: https://www.jdpowervalues.com/article/early-february-auctions-suggest-increase-number-available-late-model-trucks (Content-Type: text/html; charset=UTF-8)


Timeout downloading https://cts.businesswire.com/ct/CT?id=smartlink&url=https%3A%2F%2Fdiscover.jdpa.com%2Fhubfs%2FFiles%2FIndustry%2520Campaigns%2FValuation%2520Services%2F11.2021_Commercial%2520Truck%2520Guidelines.pdf&esheet=52535508&newsitemid=20211122005179&lan=en-US&anchor=Download+the+free+November+2021+Commercial+Truck+Guidelines+Industry+Update&index=3&md5=36ab506cfaf88d1690474e5410207909: Timeout: Request took longer than 30 seconds


Timeout downloading https://cts.businesswire.com/ct/CT?id=smartlink&url=https%3A%2F%2Fdiscover.jdpa.com%2Fhubfs%2FFiles%2FIndustry%2520Campaigns%2FValuation%2520Services%2F10.2021_Commercial%2520Truck%2520Guidelines.pdf&esheet=52513887&newsitemid=20211022005121&lan=en-US&anchor=Download+the+free+September+2021+Commercial+Truck+Guidelines+Industry+Update&index=3&md5=2228572ff53d6fd11998588f421fd73b: Timeout: Request took longer than 30 seconds


...

Timeout downloading https://cts.businesswire.com/ct/CT?id=smartlink&url=https%3A%2F%2Fdiscover.jdpa.com%2Fhubfs%2FFiles%2FIndustry%2520Campaigns%2FValuation%2520Services%2F04.2021_Commercial%2520Truck%2520Guidelines.pdf&esheet=52413430&newsitemid=20210416005556&lan=en-US&anchor=April+2021+Commercial+Truck+Guidelines+Industry+Update&index=1&md5=5715fb776dd24b82ca6ad6f324c0dec9: Timeout: Request took longer than 30 seconds


Timeout downloading https://cts.businesswire.com/ct/CT?id=smartlink&url=https%3A%2F%2Fdiscover.jdpa.com%2Fhubfs%2FFiles%2FIndustry%2520Campaigns%2FValuation%2520Services%2F03.2021_Commercial%2520Truck%2520Guidelines.pdf&esheet=52397789&newsitemid=20210318005144&lan=en-US&anchor=March+2021+Commercial+Truck+Guidelines+Industry+Update&index=1&md5=acd013843d2b7219b23940d7e4044ae1: Timeout: Request took longer than 30 seconds


Timeout downloading https://cts.businesswire.com/ct/CT?id=smartlink&url=https%3A%2F%2Fdiscover.jdpa.com%2Fhubfs%2FFiles%2FIndustry%2520Campaigns%2FValuation%2520Services%2F02.2021_Commercial%2520Truck%2520Guidelines.pdf&esheet=52381780&newsitemid=20210218006053&lan=en-US&anchor=February+2021+Commercial+Truck+Guidelines+Industry+Update&index=1&md5=4d60da057274268698fd158e1ad95aeb: Timeout: Request took longer than 30 seconds


Timeout downloading https://cts.businesswire.com/ct/CT?id=smartlink&url=https%3A%2F%2Fdiscover.jdpa.com%2Fhubfs%2FFiles%2FIndustry%2520Campaigns%2FValuation%2520Services%2F01.2021_Commercial%2520Truck%2520Guidelines.pdf&esheet=52365521&newsitemid=20210121005693&lan=en-US&anchor=January+2021+Commercial+Truck+Guidelines+Industry+Update&index=1&md5=26fe151f26df5c13b3b32604963a7331: Timeout: Request took longer than 30 seconds


Downloaded 83 PDFs.
Skipped 19 duplicate links.
Disregarded 13 non-PDF links.
Total runtime: 5.1 minutes

Log files created in C:\Users\clint\Desktop\Scraping Task\logs:
- Non-PDF links: non_pdf_links_20250328_190633.csv
- Duplicate links: duplicate_links_20250328_190633.csv
- Error links: error_links_20250328_190633.csv
- Summary: download_summary_20250328_190633.csv
